In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import matplotlib.pyplot as plt

In [2]:
engine = create_engine("sqlite:///hashtag2020.db", echo=True)
conn = engine.connect()

2021-02-04 19:36:43,748 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-04 19:36:43,749 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 19:36:43,752 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-04 19:36:43,753 INFO sqlalchemy.engine.base.Engine ()


In [3]:
# #Create the table. If we use automap_base, the table must have a primary key
# conn.execute("""
#     CREATE TABLE IF NOT EXISTS tweet_counts (
#         created_at PRIMARY KEY
#         tweet VARCHAR(280)
#         likes integer
#         retweet_count integer
#         source VARCHAR(30)
#         user_location VARCHAR(30)
#         lat integer
#         long integer
#     )
# """)

In [4]:
trump_csv_path = "../hashtag_donaldtrump.csv"
biden_csv_path = "../hashtag_joebiden.csv"

In [5]:
trump_df = pd.read_csv(trump_csv_path, lineterminator='\n')
biden_df = pd.read_csv(biden_csv_path, lineterminator='\n')

In [6]:
trump_df.head(3)

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,331617619.0,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8436472.0,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060


In [7]:
##Remove columns we don't want and set to new variable
trump_data_df = trump_df[['created_at', 'tweet', 'likes', 'retweet_count', 'source','user_location', 'lat', 'long', 'city','state','state_code','country']]
trump_data_df.head(3)

,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country
0,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,Florida,FL,United States of America
1,2020-10-15 00:00:01,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America


In [8]:
##Create index for primary key
trump_data_df.reset_index(inplace=True)
trump_data_df.head()

,index,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country
0,0,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,Florida,FL,United States of America
1,1,2020-10-15 00:00:01,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America
3,3,2020-10-15 00:00:02,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,Washington DC,38.894992,-77.036558,Washington,District of Columbia,DC,United States of America


In [9]:
##Convert NaNs into Not Provided under user location columns
str_cols = ['user_location', 'city', 'state', 'state_code', 'country']
trump_data_df.loc[:, 'lat'] = trump_data_df.loc[:, 'lat'].fillna(91)
trump_data_df.loc[:, 'long'] = trump_data_df.loc[:, 'long'].fillna(181)
trump_data_df.loc[:, str_cols] = trump_data_df.loc[:, str_cols].fillna("Not Provided")
trump_data_df.head()

/Users/buckleyweglarz/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/Users/buckleyweglarz/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


,index,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country
0,0,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,Not Provided,Florida,FL,United States of America
1,1,2020-10-15 00:00:01,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,Not Provided,91.000000,181.000000,Not Provided,Not Provided,Not Provided,Not Provided
2,2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America
3,3,2020-10-15 00:00:02,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,Not Provided,91.000000,181.000000,Not Provided,Not Provided,Not Provided,Not Provided
4,4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,Washington DC,38.894992,-77.036558,Washington,District of Columbia,DC,United States of America


In [10]:
#trump_data_df.describe()
##How to create dataframe that displays all rows that have not provided in them
#trump_data_df.loc[trump_data_df['user_location'] == 'Not Provided']

In [11]:
#trump_data_df.isnull().values.sum()

In [12]:
##Remove all cells that contain Nan
trump_dropna_df = trump_data_df.dropna(how="all")
trump_dropna_df.head(3)

,index,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country
0,0,2020-10-15 00:00:01,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,Not Provided,Florida,FL,United States of America
1,1,2020-10-15 00:00:01,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,Not Provided,91.000000,181.000000,Not Provided,Not Provided,Not Provided,Not Provided
2,2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America


In [13]:
##Split 'created_at' column into a Date and Time column
trump_dropna_df['created_at'] = pd.to_datetime(trump_dropna_df['created_at'])
trump_dropna_df['date'] = trump_dropna_df['created_at'].dt.strftime('%m')
trump_dropna_df['time'] = trump_dropna_df['created_at'].dt.strftime('%H:%M')

In [14]:
trump_dropna_df.date.value_counts()

11    588735
10    382184
Name: date, dtype: int64

In [15]:
# df.loc[df['Population(B)'] < 1] = 0
# df


trump_dropna_df.loc[trump_dropna_df['date'] < 10] = 'Oct'
#trump_dropna_df.loc[(trump_dropna_df.date == 11), 'date'] = 'Nov'

#

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
##Drop 'created_at' column
trump_clean_df = trump_dropna_df.drop(columns=['created_at'])
trump_clean_df.head(3)
trump_dropna_df.date.value_counts()

In [ ]:
#Try grouping dates by month
x = trump_clean_df.date
y = trump_clean_df.likes

In [ ]:
#Plotting bar graph to show how many tweets by day
plt.plot(x, y)
plt.show()

In [ ]:
# group_df = trump_dropna_df["date"].value_counts()
# group_df = trump_dropna_df.sort_values(by='date', ascending=False)
# group_df.plot(kind="bar", title="Trump Tweet Frequency")

In [ ]:
group_df = trump_dropna_df[likes"].value_counts()
group_df.plot(kind="bar", title="Trump Tweet Like Freq.")

# trump_dropna_df.dtypes

In [ ]:
# Insert data into DB
df.to_sql("tweet_counts", conn, if_exists="append", index=False)